## Toggle Flip-Flop

In this example we create a toggle flip-flop (TFF) from a d-flip-flop and an xor gate. This shows how to construct sequential logic circuits using `Magma`.

In [ ]:
import magma as m

import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import DFF

import lattice ice40


In [ ]:
def tff():
    # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
    ff = DFF()

    # compute the next state as the not of the old state ff.O
    ff( ~ff.O )

    return ff.O

In [ ]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J3[0].rename('J3').output().on()

main = icestick.DefineMain()
m.wire( tff(), main.J3 )
m.EndDefine()

## Compile and Build

In [ ]:
m.compile("build/tff", main)

In [ ]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif tff.blif' tff.v
arachne-pnr -q -d 1k -o tff.txt -p tff.pcf tff.blif 
icepack tff.txt tff.bin
# iceprog tff.bin

The pin at J3[0] should now be toggling.